# Part 1: Calculating_ETc_eff

This part of the code estimates the crop evapotranspiration under standard conditions (ETc) and single crop coefficient (kc)

## First import modules required

In [3]:
import pandas as pd
from datetime import datetime
import pyeto
import numpy as np
import ast       
import math
from ast import literal_eval  
from pandas import DataFrame
from scipy.interpolate import interp1d
import dateutil     #dateutil module provides powerful extensions to the standard datetime module
from dateutil import parser  #This module offers reads the given date in string and convert it to date format or timestamps,it represent a date and/or time from most known formats 

# note that pyeto is available here https://github.com/woodcrafty/PyETo.git
from pyeto import fao

%matplotlib inline

math.exp = np.exp
math.pow = np.power
math.sqrt = np.sqrt

## Importing initial dataset 

Main inputs include (TBU):

* Country name
* Crop lat, lon
* Crop area (m2)
* Ground water depth (m)
* Elevation (m)
* Average wind speed per month (m s-1)
* Min, Average, Max temperature per month (C)
* Average precipitation per month mm)
* Average solar irradiation per month (kJ m-2 day-1)

In [4]:
#This is a sample dataset with about 1000 rows located in Zambezia province in Mozambique. 
df=pd.read_csv('Pilot_Input_Crop_Cassava.csv') 

# Cleaning data commands

#df=df.drop(df[df['harv_t']==0].index) #Deleting any point that has zero harvesting area
#df=df.reset_index()    #reseting the index after deleting the zero value points
#del df['index']        #The previous step will generate a new column form the old index, this step deletes this column with old index

#in the intital code, there are two more steps related to calibration and projection. Check if you need them
#define available water content  ##Not used in the ETO estimation

#where 0.9 rooting depth for maize and 50% maximum depletion factor

#def awc_class(row):
#    if (row['awc_class']==0):
#        return 0
#    elif (row['awc_class']==1):
#        return 150*0.9*0.5
#    elif (row['awc_class']==2):
#        return 125*0.9*0.5
#    elif (row['awc_class']==3):
#        return 100*0.9*0.5
#    elif (row['awc_class']==4):
#        return 75*0.9*0.5
#    elif (row['awc_class']==5):
#        return 50*0.9*0.5
#    elif (row['awc_class']==6):
#        return 15*0.9*0.5
#    elif (row['awc_class']==7):
#        return 0*0.9*0.5
#    else:
#        return 75*0.9*0.5

#df['awc'] = df.apply(awc_class,axis=1)

df.head(5)

,TOTALAREA,area_m2,MAIZEAREA,long,lat,FID,ID,ORIG_FID,gw_depth,TDS,...,tmin_3,tmin_4,tmin_5,tmin_6,tmin_7,tmin_8,tmin_9,tmin_10,tmin_11,tmin_12
0,323,3.342524e+05,200.058533,36.708337,-15.04167,1,374005,0,37,3000,...,10.0161,14.8924,19.814301,24.798201,28.101999,27.357500,23.091101,16.896299,9.71018,5.52086
1,124,4.027141e+05,22.442619,36.791670,-15.04167,2,374006,1,34,3000,...,10.0161,14.8924,19.814301,24.798201,28.101999,27.357500,23.091101,16.896299,9.71018,5.52086
2,195,7.752338e+05,34.362381,36.875003,-15.04167,3,374010,2,29,3000,...,10.0390,14.9651,19.886801,24.865999,28.084400,27.346201,23.111000,16.821600,9.63013,5.43547
3,291,1.205407e+06,28.202349,36.958337,-15.04167,4,374017,3,43,3000,...,10.0390,14.9651,19.886801,24.865999,28.084400,27.346201,23.111000,16.821600,9.63013,5.43547
4,402,1.658504e+06,88.700356,37.125003,-15.04167,5,374021,4,17,3000,...,10.0761,14.9526,19.824699,24.807501,28.113001,27.369101,23.100300,16.906200,9.71928,5.52977


### Estimating ETo based on FAO-56 Penman-Monteith formula

In [5]:
%%time
# Defining function

def evap_i(lat,elev,wind,srad,tmin,tmax,tavg,month):
    if month ==1:
        J = 15
    else:
        J = 15 + (month-1)*30
        
    latitude = pyeto.deg2rad(lat)
    atmosphericVapourPressure = pyeto.avp_from_tmin(tmin)
    saturationVapourPressure = pyeto.svp_from_t(tavg)
    ird = pyeto.inv_rel_dist_earth_sun(J)
    solarDeclination = pyeto.sol_dec(J)
    sha = [pyeto.sunset_hour_angle(l, solarDeclination) for l in latitude]
    extraterrestrialRad = [pyeto.et_rad(x, solarDeclination,y,ird) for x, y in zip(latitude,sha)]
    clearSkyRad = pyeto.cs_rad(elev,extraterrestrialRad)
    netInSolRadnet = pyeto.net_in_sol_rad(srad*0.001, albedo=0.23)
    netOutSolRadnet = pyeto.net_out_lw_rad(tmin, tmax, srad*0.001, clearSkyRad, atmosphericVapourPressure)
    netRadiation = pyeto.net_rad(netInSolRadnet,netOutSolRadnet)
    tempKelvin = pyeto.celsius2kelvin(tavg)
    windSpeed2m = wind
    slopeSvp = pyeto.delta_svp(tavg)
    atmPressure = pyeto.atm_pressure(elev)
    psyConstant = pyeto.psy_const(atmPressure)
    
    return pyeto.fao56_penman_monteith(netRadiation, tempKelvin, windSpeed2m, saturationVapourPressure, atmosphericVapourPressure, slopeSvp, psyConstant, shf=0.0)

#Initiate
for i in range(1,13):
    df['ETo_{}'.format(i)]=0  ##To make sure the it is reset to zero

# calculate ETo for each row for each month 
# range(1,13) and .format(i): to generate monthly calculation of ETo
# df.iterrows() and use of .iloc[index]: To make sure the calculation will be repearted for each index point. 
for i in range(1,13):
    df['ETo_{}'.format(i)] = evap_i(df['lat'],df['elevation'],df['wind_{}'.format(i)],df['srad_{}'.format(i)],df['tmin_{}'.format(i)],df['tmax_{}'.format(i)],df['tavg_{}'.format(i)],i)


### You can save the ETO to save time and avoid computing it everytime

## Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('Pilot_ETO.xlsx', engine='xlsxwriter')
## Convert the dataframe to an XlsxWriter Excel object.
#df.to_excel(writer, sheet_name='ETO_all')
## Close the Pandas Excel writer and output the Excel file.
#writer.save()

Wall time: 255 ms


### Calculate effective rainfall for every row for each month

##### TODO list

- [X] Fill in the sources for effective rainfall formulae - issue #1
- [ ] Review & Update effective rainfall formula - issue #1

--------------------------------------------------------------------------------------------------------------

**Sources:**
 - USDA‐SCS Method, Chapter 2 "Irrigation Water Requirements", page 147, url:https://www.wcc.nrcs.usda.gov/ftpref/wntsc/waterMgt/irrigation/NEH15/ch2.pdf
 - FAO, "Effective rainfall in irrigated agriculture", emperical methods, url: http://www.fao.org/3/x5560e/x5560e00.htm#Contents
 - M. Ali, S. Mubarak, "Effective Rainfall Calculation Methods for Field Crops: An Overview, Analysis and New Formulation", url: https://doi.org/10.9734/ARJA/2017/36812
 - FAO, "Irrigation Water Management: Irrigation Water Needs", url: http://www.fao.org/3/S2022E/s2022e00.htm#Contents
 - S. Mohan, B. Simhadrirao, N. Arumugam, "Comparative study of effective rainfall estimation methods for lowland rice", url: https://link.springer.com/article/10.1007/BF00698810
 - Balram Panigrahi, Megh R. Goyal, (Book), "Soil and Water Engineering: Principles and Applications of Modeling", page 265, url: https://books.google.se/books?id=wR9jDAAAQBAJ&pg=PA264&lpg=PA264&dq=criwar+semi+empirical+information+Pe&source=bl&ots=Bim8aJhvm3&sig=ACfU3U3xQK7FeZxIutU1-W962mydu3yesQ&hl=en&sa=X&ved=2ahUKEwjH0e3H-JriAhUxtIsKHftyB_UQ6AEwCnoECAcQAQ#v=onepage&q=criwar%20semi%20empirical%20information%20Pe&f=false


In [6]:
%%time

#Define rainfall function 
def eff_rainfall(prec,eto):
    return (1.253*((prec**0.824)-2.935))*10**(0.001*eto)  

#Initiate
for i in range(1,13):
    df['eff_{}'.format(i)]=0
    
for i in range(1,13):
    df.loc[df['prec_{}'.format(i)] < 12.5, 'eff_{}'.format(i)] = df['prec_{}'.format(i)]/30
    df.loc[df['prec_{}'.format(i)] >= 12.5, 'eff_{}'.format(i)] = eff_rainfall(df['prec_{}'.format(i)],df['ETo_{}'.format(i)])/30 
    
### You can save the ETO_RF to save time and avoid computing it everytime

##Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('Pilot_ETO_RF.xlsx', engine='xlsxwriter')
## Convert the dataframe to an XlsxWriter Excel object.
#df.to_excel(writer, sheet_name='RF_all')
## Close the Pandas Excel writer and output the Excel file.
#writer.save()

Wall time: 121 ms


### Define rainfall pattern (unimodal vs bimodal vs trimodal)

##### TODO list

- [ ] Identify agroecological zones in Mozambique - issue #6
- [ ] Decide on crops we are modelling in the pilot case study - issue #6
- [ ] Specify crop calendar per region and crop - issue #6
- [ ] Modify input files as needed to reflect the above - issue #6

--------------------------------------------------------------------------------------------------------------

In [7]:
#for this example we assume that unimodal pattern for the whole study area, which means it has one raining season only. 
df['Mode']=('region_1')

### Calculate kc based on the growing stage (month - planting, growing, harvesting season/month)

##### TODO list

- [ ] Merge Li1, Li2 and kci1, kci2 into  - issue #8

--------------------------------------------------------------------------------------------------------------

In [8]:
# Define kc function and its attributes

def kc(plantation,Li1,Li2,Ld,Lm,Le,kci1,kci2,kcd,kcm,kce,isodate):  #new code: Li1, Li2, kci1 and kci2 
    
    """
Each crop goes through four growing stages: initial - development - mid-season and end-season (check FAO-56 chapter 6 for more details)

Inputs:
Plantation = plantation datetime 
Li = length of the initial stage (in days)
Ld = length of the development stage (in days)
Lm = length of the mid-season stage (in days)
Le = length of the end-season stage (in days)

kci = crop coefficient 'kc' at the initial stage. In this stage the ckc value is constant and equal to kci
kcm = crop coefficient 'kc' at the mid-season stage.  In this stage the ckc value is constant and equal to kcm
kce = crop coefficient 'kc' at the end-season stage. In this stege the ckc value varies linearly between kce and kcm. check equation 66 (page 132, FAO56). 
isodate = current date (optional)

Outputs: 
* ckc : current crop coefficient, which is constant in the initial and mid-season stages and varies linearly in the development (increasing) and end-season (declining) stages. 

Some Examples:
     Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-01-20")
        >>> 0.15
     
     Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-02-10")
        >>> 0.774
     
     Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-03-12")
        >>> 1.19
     
     Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-04-06")
        >>> 0.559
    
    """
#step 1: 
    
    plantation = pd.to_datetime(plantation, format='%d/%m') #converting the plantation input info to data time
    isodate = pd.to_datetime(isodate , format='%d/%m')  #converting the current date input info to data time
    test = ((isodate-plantation).days)%365   #The difference in days between the current day and the plantation day.
    
    # Setting the plantation date and the current date (this is not used)
    Jc = test   
    Jp = 0
    J = (Jc - Jp)%365  # %365 means the remaing days of the year
    
#Step 2: Calculating the day of the year when each crop stage ends placing the date in the number of days year betweem 0 (1/jan) and 365 (31/Jan)
    JLi1 = Jp + Li1    #end of initial stage = plantation date + lenght of initial stage
    JLi2 = JLi1 + Li2
    JLd = JLi2 + Ld   #end of development stage = end of initial stage + length of development stage
    JLm = JLd + Lm   #end of mid-season stage = end of development stage + length of mid-season stage
    JLe = JLm + Le   #end of end-season stage = end of mid-season stage + length of end-season stage

#step 3: calculating ckc based on the end of each stage date

    if Jc > Jp and Jc < JLe:   #if the current date is greater than the plantation date and it is greater than the end of end-season stage
        if J <= JLi1:    
            ckc = kci1  #if the current date is before the end of initial stage then ckc = kci the coefficient of the initial stege
        elif Jc > JLi1 and Jc <=JLi2: #New: to account for two init stages
            ckc = kci2
        elif Jc > JLi2 and Jc <=JLd:  #if the current date is betweeen the end of the intial stege and the end of the development stage, then ckc is computed based on equation 66 (page 132.FAO56)
            ckc = kci2 + ((Jc-JLi2)/Ld * (kcm-kci2))
        elif Jc > JLd and Jc <= JLm: 
            ckc = kcm
        elif Jc > JLm and Jc <= JLe:
            ckc = kcm + ((Jc-JLm)/Le * (kce-kcm))
            
    else:
        ckc = 0
    
    return ckc

In [9]:
%%time

# Import sample crop calendar
mode = pd.read_excel('Pilot_Input_Crop_Calendar.xlsx')

#Note: The code here is adjusted to avoid the end of year issue. In other cases, the init1 and init2 are one stage init:
#pay attention to all changes, you may need to change this if the crop calendar change 

#Planting season: Initial Stage 1 (plant = init1+ init2 )
init1_start = pd.to_datetime(mode['init1_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
init1_end = pd.to_datetime(mode['init1_end'], format='%d/%m')
mode['init1_start_month'] = init1_start.dt.month
mode['init1_end_month'] = init1_end.dt.month
mode['init1_days'] = abs(init1_end - init1_start).dt.days #Calculating the length of the planting season
Li1 = abs(init1_end - init1_start).dt.days

#Planting season: Initial Stage 2 (plant = init1+ init2 )
init2_start = pd.to_datetime(mode['init2_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
init2_end = pd.to_datetime(mode['init2_end'], format='%d/%m')
mode['init2_start_month'] = init2_start.dt.month
mode['init2_end_month'] = init2_end.dt.month
mode['init2_days'] = abs(init2_end - init2_start).dt.days #Calculating the length of the planting season
Li2 = abs(init2_end - init2_start).dt.days

#growing 1: Development Stage (grow = dev)
dev_start = pd.to_datetime(mode['dev_start'], format='%d/%m')
dev_end = pd.to_datetime(mode['dev_end'], format='%d/%m')
mode['dev_start_month'] = dev_start.dt.month
mode['dev_end_month'] = dev_end.dt.month
mode['dev_days'] = abs(dev_end - dev_start).dt.days
Ld = abs(dev_end - dev_start).dt.days 

#growing 2: Mid stage ( add : mid)
mid_start = pd.to_datetime(mode['mid_start'], format='%d/%m')
mid_end = pd.to_datetime(mode['mid_end'], format='%d/%m')
mode['mid_start_month'] = mid_start.dt.month
mode['mid_end_month'] = mid_end.dt.month
mode['mid_days'] = abs(mid_end - mid_start).dt.days
Lm = abs(mid_end - mid_start).dt.days 

#Harvesting: Late stage (harv = late)
late_start = pd.to_datetime(mode['late_start'], format='%d/%m') #defining the plant start date from excil and setting the correct month and days sequence to read.
late_end = pd.to_datetime(mode['late_end'], format='%d/%m')
mode['late_start_month'] = late_start.dt.month
mode['late_end_month'] = late_end.dt.month
mode['late_days'] = abs(late_end - late_start).dt.days #Calculating the length of the planting season
Le = abs(late_end - late_start).dt.days

for i in range(1,13):
    mode['kc_{}'.format(i)]=0
    
for index,row in mode.iterrows():
    for i in range(0,12):
        init1_start = pd.to_datetime(mode['init1_start'].iloc[index], format='%d/%m') #read the plant start date from excel. 
        day_start= (init1_start.day+1-31)%31   #what does this represent??   
        
        if (init1_start.day-1==30):
            month_start = (init1_start.month+1-12)%12  #next month
        else:
            month_start = (init1_start.month-12)%12  #the current month
            
        month_start = (month_start+i)%12
        if (month_start==0):
            month_start = 12
        mode.loc[index,'kc_{}'.format(month_start)] = kc(mode['init1_start'].iloc[index],mode['init1_days'].iloc[index],mode['init2_days'].iloc[index],mode['dev_days'].iloc[index],mode['mid_days'].iloc[index],mode['late_days'].iloc[index],0.8,0.8,0.9,1,0.8,'{}/{}'.format(day_start,month_start))
        #print (kc)
        
# so far we worked with (df) dataframe which contains GIS outputs, then we created a (mode) dataframe. 
# Here we merge them on into one new dataframe called (data) and we chose the merging to be on the 'Mode' column 

data = pd.merge(df,mode,on='Mode') 

# Calculating the annual precipitation: which is the sum of precipitation values
data['precipitation_annual']=data.filter(like='prec_').sum(axis=1)  #Filter is used to specify the column of interest

# Finally, print results of part 1

#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Pilot_Result_Part1.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
data.to_excel(writer, sheet_name='Total_area')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

Wall time: 3.75 s
